In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [ ]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit()

In [ ]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=4) + ns(VehAge, df=4) + ns(DrivAge, df=4) + ns(VehPower, df=4) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21262.98, loss=0.309547, deviance=125811.8, log_likelihood=-83679.72, aic=167443.45, gini=0.284, auc=0.642, ae_ratio=1.0068, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1926.11, actual=0.389385, predicted=0.075199, ae_ratio=5.1781), DecileMetrics(decile=2, n=40643, exposure=4241.39, actual=0.224455, predicted=0.088655, ae_ratio=2.5318), DecileMetrics(decile=3, n=40643, exposure=8285.17, actual=0.163666, predicted=0.084391, ae_ratio=1.9394), DecileMetrics(decile=4, n=40643, exposure=14062.94, actual=0.119534, predic

In [ ]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139175.1825 Deviance:                209350.2188
AIC:                     278434.3650 Null Deviance:           220244.0296
BIC:                     278914.2956 Pearson chi2:             1522762.08
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -3.0925 

In [ ]:
result.predict(holdout_data)

array([0.05194827, 0.04859354, 0.04948878, ..., 0.0055668 , 0.01744882,
       0.07106865], shape=(135957,))

In [ ]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [ ]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84123.8454 Deviance:                126700.0411
AIC:                     168269.6908 Null Deviance:           132328.3090
BIC:                     168389.7579 Pearson chi2:              934829.08
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.3801     0.1556  -28.156  <0.0001   [ -4.6850,  -4.0752]  ***
BonusMalus             0.0219     0.0004   54.643  

In [ ]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [ ]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21362.12, loss=0.31169, deviance=126682.94, log_likelihood=-84115.29, aic=168252.59, gini=0.2597, auc=0.6298, ae_ratio=1.0021, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1902.18, actual=0.411108, predicted=0.077852, ae_ratio=5.2806), DecileMetrics(decile=2, n=40643, exposure=4172.41, actual=0.238711, predicted=0.091277, ae_ratio=2.6152), DecileMetrics(decile=3, n=40643, exposure=8103.17, actual=0.164627, predicted=0.087369, ae_ratio=1.8843), DecileMetrics(decile=4, n=40643, exposure=13723.56, actual=0.127081, predicted=0.082915, ae_ratio=1.5327), DecileMetrics(decile

In [ ]:
predictions = result.predict(holdout_data)

In [ ]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [ ]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [ ]:
predictions2 = loaded.predict(holdout_data)

In [ ]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [ ]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21362.12, loss=0.31169, deviance=126682.94, log_likelihood=-84115.29, aic=168252.59, gini=0.2597, auc=0.6298, ae_ratio=1.0021, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1902.18, actual=0.411108, predicted=0.077852, ae_ratio=5.2806), DecileMetrics(decile=2, n=40643, exposure=4172.41, actual=0.238711, predicted=0.091277, ae_ratio=2.6152), DecileMetrics(decile=3, n=40643, exposure=8103.17, actual=0.164627, predicted=0.087369, ae_ratio=1.8843), DecileMetrics(decile=4, n=40643, exposure=13723.56, actual=0.127081, predicted=0.082915, ae_ratio=1.5327), DecileMetrics(decile

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [ ]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21411.99, loss=0.318664, deviance=129517.45, log_likelihood=-85532.55, aic=171073.1, gini=0.1823, auc=0.5911, ae_ratio=0.9998, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1694.99, actual=0.640122, predicted=0.088941, ae_ratio=7.1972), DecileMetrics(decile=2, n=40643, exposure=3748.79, actual=0.287826, predicted=0.099125, ae_ratio=2.9037), DecileMetrics(decile=3, n=40643, exposure=7524.67, actual=0.182865, predicted=0.094958, ae_ratio=1.9257), DecileMetrics(decile=4, n=40643, exposure=12522.85, actual=0.138706, predicted=0.094971, ae_ratio=1.4605), DecileMetrics(decile=5, n=40643, exposure=18495.93, act

In [ ]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [ ]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=4) + ns(DrivAge, df=4) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21427.49, 'loss': 0.310817, 'deviance': 126327.96, 'log_likelihood': -83937.8, 'aic': 167919.61, 'gini': 0.2737, 'auc': 0.6368, 'ae_ratio': 0.9991, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1903.66, 'actual': 0.398181, 'predicted': 0.077433, 'ae_ratio': 5.1423}, {'decile': 2, 'n': 40643, 'exposure': 4162.08, 'actual': 0.239063, 'predicted': 0.090846, 'ae_ratio': 2.6315}, {'decile': 3, 'n': 40643, 'exposure': 8133.38, 'actual': 0.164384, 'predicted': 0.086962, 'ae_ratio': 1.8903}, {'decile': 4, 'n': 40643, 'exposure': 13715.88, 'actual': 0.123652, 'predicted': 0.082664, 'ae_ratio': 1.4958}, {'decile': 5, 'n': 4064